In [ ]:
# Deep Learning Methods #3

# Fine-Tuning
from tensorflow.keras.applications import Xception, ResNet50, EfficientNetB4
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

def build_finetuned_model(base_model, input_shape):
    base_model.trainable = False

    inputs = base_model.input
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, x)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (224, 224, 3)
xception_base = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
efficientnet_base = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)

xception_model = build_finetuned_model(xception_base, input_shape)
resnet_model = build_finetuned_model(resnet_base, input_shape)
efficientnet_model = build_finetuned_model(efficientnet_base, input_shape)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Initial training of new layers
history = xception_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Unfreeze some layers in the backbone model
for layer in xception_model.layers[-20:]:
    layer.trainable = True

xception_model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tuning the whole model
history_finetune = xception_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)